# Audio Transcription and Summarization with Whisper and OpenAI

## Description
This program is a tool for automatically transcribing and summarizing the content of audio and video files. It uses Whisper's voice recognition technology for transcription and OpenAI's API for generating summaries. It is designed to be run in a Google Colab environment.

## Features
- **Audio/Video Transcription**: Uses the Whisper model to transcribe audio extracted from video files.
- **Automatic Language Detection**: Ability to detect the language of the audio for accurate transcription.
- **Summary Generation**: Uses the OpenAI API to summarize the transcribed text, ideal for meetings, conferences, etc.

## Use of the OpenAI API
This program makes use of the OpenAI API, which is a paid service. Please **use this API responsibly** to avoid excessive charges.

## GPU Configuration
It's crucial to ensure that you are using a GPU in Google Colab for optimal performance. To activate the GPU:
   - Go to `Runtime` > `Change runtime type`.
   - Select `T4 GPU` in the hardware accelerator option.
   - Save and proceed to run the notebook.

## Usage Instructions
1. **Video Upload**: Upload your video file to the Colab environment.
2. **Video Configuration**: Update the path in the script to match the location of your uploaded video.
3. **API Configuration**: Verify that the OpenAI API key is correctly configured in the environment.
4. **Script Execution**: Go to `Runtime` > `Run all`.
5. **Access to Results**: The transcribed and summarized texts will automatically be saved in text files within the Colab environment.

## Dependencies
- **Whisper**: Advanced voice recognition.
- **moviepy**: Video editing and processing.
- **ffmpeg**: Conversion and manipulation of multimedia files.
- **tiktoken**: Authentication token management.
- **openai**: Interface for OpenAI language models (GPT-3-5 in this case).

## Authors and Acknowledgments
This script has been developed by Alejandro Oliveros Ordás with the help of OpenAI and Whisper technologies.


In [ ]:
import os


os.environ['OPENAI_API_KEY'] = #your api key

video_path = #your video path

In [ ]:
import torch

# Check if CUDA is available for GPU use
torch.cuda.is_available()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


In [ ]:
# @title Installing Dependencies
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git
!pip install moviepy
!apt install ffmpeg
!pip install tiktoken
!pip install openai

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-o46h1hkh
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-o46h1hkh
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802825 sha256=e878f0edec8c072ef02dc416f9af4ea7c31314510f1a493d8297126a8841d115
  Stored in directory: /tmp/pip-ephem-wheel-cache-8mwt3s4y/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
  Attempting uninstall: openai-whisper
    Found existing installation: openai-whisper 20231117
    Uninstalling openai-whisper-20231117:
      Successfully uninstalled openai-whisper-20231117
Reading package lists... Done
Building dep

In [ ]:
# @title Import Dependencies
import whisper
import numpy as np
import subprocess
import io
import requests
from openai import OpenAI


In [ ]:
# @title Load the Whisper model

# Cargar el modelo de Whisper
model = whisper.load_model("medium", device=DEVICE)
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|██████████████████████████████████████| 1.42G/1.42G [00:15<00:00, 100MiB/s]


Model is multilingual and has 762,321,920 parameters.


In [ ]:
# @title Upload video and extract audio
# Cargar el video y extraer el audio usando ffmpeg
extracted_audio_path = '/content/extracted_audio.mp3'  # Ruta para el archivo de audio extraído
command = ['ffmpeg', '-i', video_path, '-f', 'mp3', extracted_audio_path]
subprocess.run(command)

CompletedProcess(args=['ffmpeg', '-i', '/content/drive/MyDrive/MyBS/wf_rotado_no_va.mkv', '-f', 'mp3', '/content/extracted_audio.mp3'], returncode=0)

In [ ]:
# @title Process audio and detect language

# Procesar el audio
audio = whisper.load_audio(extracted_audio_path)
audio = whisper.pad_or_trim(audio)
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# Detectar el idioma del audio
_, probs = model.detect_language(mel)
detected_language = max(probs, key=probs.get)
print(f"Detected language: {detected_language}")

Detected language: es


In [ ]:
# @title Transcribe with Whisper and save output to transcription.txt

# Opciones para la transcripción utilizando el idioma detectado
options = whisper.DecodingOptions(language=detected_language, without_timestamps=True, fp16=False)
result = whisper.decode(model, mel, options)

result = model.transcribe(extracted_audio_path)
print(result["text"])

# Guardar la transcripción en .txt
result_text = result["text"]
file_path = '/content/transcription.txt'

with open(file_path, 'w') as file:
    file.write(result_text)

print(f'Result of transcription text saved to: {file_path}')

 a ver lo que he visto esta no es la que decía yo la que acaban que en ese dice el proceso el proceso derrotado de folder a pe folder como que no arranca vale este es el proceso que pasa los desarrolladores dejan sus cosas en carpeta e y automáticamente se pasa carpeta p si ha ido bien vale entonces si no salta suele ser porque el proceso que se encarga de ello que está aquí en p y m utilidades vale este rotado carpetas por lo que sea mejor se ha quedado pillado en teoría yo lo que he hecho ha sido darle a un restar al proceso que ya ha acabado al cliente le puede decir que ya ha acabado pero hay que ver es que pilla la planificación porque si ponen otro vamos a estar en la misma vale entonces el rollo es aquí no es que duro pues te vienes aquí haces un versión check out para poder manipular el workflow vale te vas a workflow ed y schedule es schedule aquí schedule y aquí viene la digamos cuando como ejecutarse entonces después de ponerle que empiece yo no sé por qué supone que esto no

In [ ]:
# @title Summarise the transcript with the GPT-3.5 API and save the output to summary_transcription.txt

def summarize_text(text):
    client = OpenAI(
        api_key=os.environ.get("OPENAI_API_KEY"),
    )

    chat_completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages = [
    {
        "role": "system",
        "content": "Tu tarea es generar un resumen completo y claro de reuniones de trabajo transcritas. El resumen debe incluir los puntos clave discutidos, las decisiones tomadas y cualquier acción o tarea asignada. Evita detalles innecesarios y enfócate en la esencia de la reunión. El resumen debe ser lo suficientemente informativo para alguien que no asistió a la reunión."
    },
    {
        "role": "user",
        "content": "Por favor, resume la siguiente transcripción de la reunión: " + text
    }
]
    )

    # Accede a la respuesta de la siguiente manera
    resumen = chat_completion.choices[0].message.content.strip()
    return resumen

# Uso de la función de resumen
result_text = result["text"]
summary = summarize_text(result_text)

# El resumen hecho por gpt-3.5
print(summary)

# Guardar el resumen en un archivo .txt
file_path = '/content/summary_transcription.txt'
with open(file_path, 'w') as file:
    file.write(summary)

print(f'Summary text saved to: {file_path}')

La reunión tuvo lugar para discutir el proceso de rotación de carpetas en el trabajo de los desarrolladores. Se mencionó que a veces el proceso se queda pillado en la carpeta de utilidades y no se ejecuta correctamente. Se decidió reiniciar el proceso e informar al cliente una vez que haya finalizado. También se discutió el manejo de las peticiones pendientes y se acordó no dejarlas corriendo indefinidamente, sino ponerlas en pausa si es necesario. Se asignó la tarea de revisar las peticiones con el equipo y mencionar cualquier problema en nuestro chat interno. Además, se mencionó una solicitud relacionada con el tema de Jaz que se abordaría más adelante. Se compartió una carpeta con archivos sobre el proceso de rotación para que todos los participantes pudieran revisarlos y resolver cualquier duda.
Summary text saved to: /content/summary_transcription.txt
